In [5]:
organism_name = "Vibrio natriegens"

In [6]:
from Bio import Entrez
from Bio.Blast import NCBIWWW, NCBIXML

Entrez.email = "your.email@example.com"  # Required by NCBI



# Step 1: Get a representative 16S rRNA sequence for E. coli
print(f"Fetching 16S rRNA sequence for {organism_name}...")
search = Entrez.esearch(db="nucleotide", term=f"{organism_name}[Organism] AND 16S ribosomal RNA[Title]", retmax=1)
record = Entrez.read(search)
seq_id = record["IdList"][0]

fasta = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text").read()

# Step 2: Run BLAST against nt database, excluding E. coli
print("Running BLAST (excluding Escherichia coli)...")
result_handle = NCBIWWW.qblast(
    program="blastn",
    database="nt",
    sequence=fasta,
    entrez_query="NOT Escherichia coli[Organism]",
    hitlist_size=100  # get many results so we can deduplicate
)

# Step 3: Parse BLAST results
blast_record = next(NCBIXML.parse(result_handle))

unique_species = {}
for alignment in blast_record.alignments:
    title = alignment.hit_def
    hsp = alignment.hsps[0]
    identity = (hsp.identities / hsp.align_length) * 100
    score = hsp.score

    # Extract species name (usually first two words of the title)
    parts = title.split()
    if len(parts) >= 2:
        species_name = " ".join(parts[:2])
    else:
        continue

    # Skip if it's E. coli or already recorded
    if "Escherichia coli" in species_name or species_name in unique_species:
        continue

    # Keep best (highest score) alignment per unique species
    unique_species[species_name] = {
        "title": title,
        "score": score,
        "identity": identity,
        "align_length": hsp.align_length
    }

# Step 4: Sort by similarity (descending)
sorted_species = sorted(unique_species.items(), key=lambda x: x[1]["identity"], reverse=True)

print("\nTop genetically related distinct species (excluding E. coli):\n")
for species, data in sorted_species[:10]:
    print(f"{species}\n  Identity: {data['identity']:.2f}% | Score: {data['score']} | Length: {data['align_length']}\n")


Fetching 16S rRNA sequence for Vibrio natriegens...
Running BLAST (excluding Escherichia coli)...

Top genetically related distinct species (excluding E. coli):

Vibrio natriegens
  Identity: 100.00% | Score: 2814.0 | Length: 1407

Vibrio parahaemolyticus
  Identity: 100.00% | Score: 2814.0 | Length: 1407

Vibrio chemaguriensis
  Identity: 99.93% | Score: 2805.0 | Length: 1407

Vibrio sp.
  Identity: 99.93% | Score: 2805.0 | Length: 1407

Uncultured bacterium
  Identity: 99.36% | Score: 2749.0 | Length: 1411

Vibrio campbellii
  Identity: 99.00% | Score: 2744.0 | Length: 1407

Mutant Vibrio
  Identity: 99.00% | Score: 2744.0 | Length: 1407

Vibrio alginolyticus
  Identity: 98.93% | Score: 2739.0 | Length: 1407

MAG: Vibrio
  Identity: 98.93% | Score: 2739.0 | Length: 1407

MAG: uncultured
  Identity: 98.93% | Score: 2739.0 | Length: 1407



In [7]:
# Filter out 'Uncultured bacterium', entries with 'sp.', and invalid names
filtered_species = {}
for species, data in unique_species.items():
    # Skip if it contains 'Uncultured' (includes 'Uncultured bacterium', 'Uncultured prokaryote', etc.)
    if 'Uncultured' in species or 'uncultured' in species:
        continue
    # Skip if it ends with 'sp.' (e.g., "Shigella sp.", "Escherichia sp.")
    if species.endswith('sp.'):
        continue
    # Skip if it contains ' sp.'
    if ' sp.' in species:
        continue
    # Skip if it contains 'MAG:' (metagenome-assembled genomes)
    if 'MAG:' in species:
        continue
    # Skip if it contains 'Mutant'
    if 'Mutant' in species:
        continue
    # Skip if it contains 'Bacterium' without a proper species name
    if species.startswith('Bacterium '):
        continue
    # Keep the rest
    filtered_species[species] = data

# Sort by similarity (descending)
sorted_filtered = sorted(filtered_species.items(), key=lambda x: x[1]["identity"], reverse=True)

print(f"Original species count: {len(unique_species)}")
print(f"Filtered species count: {len(filtered_species)}")
print(f"\nTop genetically related distinct species (filtered):\n")
for species, data in sorted_filtered[:10]:
    print(f"{species}\n  Identity: {data['identity']:.2f}% | Score: {data['score']} | Length: {data['align_length']}\n")


Original species count: 13
Filtered species count: 11

Top genetically related distinct species (filtered):

Vibrio natriegens
  Identity: 100.00% | Score: 2814.0 | Length: 1407

Vibrio parahaemolyticus
  Identity: 100.00% | Score: 2814.0 | Length: 1407

Vibrio chemaguriensis
  Identity: 99.93% | Score: 2805.0 | Length: 1407

Vibrio campbellii
  Identity: 99.00% | Score: 2744.0 | Length: 1407

Mutant Vibrio
  Identity: 99.00% | Score: 2744.0 | Length: 1407

Vibrio alginolyticus
  Identity: 98.93% | Score: 2739.0 | Length: 1407

MAG: Vibrio
  Identity: 98.93% | Score: 2739.0 | Length: 1407

MAG: uncultured
  Identity: 98.93% | Score: 2739.0 | Length: 1407

Bacterium ST4
  Identity: 98.86% | Score: 2734.0 | Length: 1407

Vibrio neocaledonicus
  Identity: 98.86% | Score: 2734.0 | Length: 1407



In [ ]:
# Debug: Print all unique_species keys to see what we're filtering
print("All species names in unique_species:")
for species in unique_species.keys():
    print(f"  '{species}'")


Creates a list (not a list) called sorted_filtered that has the closest related organisms